<a href="https://colab.research.google.com/github/fabriciovital/spark_pyspark_analise_filmes_imdb/blob/main/Spark_pyspark_analise_filmes_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Instalar o java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [7]:
#Baixar a versão mais recente do spark
!wget -O spark-3.5.2-bin-hadoop3.tgz "https://www.apache.org/dyn/closer.lua/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz?action=download"

--2024-09-03 02:12:47--  https://www.apache.org/dyn/closer.lua/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz?action=download
Resolving www.apache.org (www.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to www.apache.org (www.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz [following]
--2024-09-03 02:12:47--  https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400822099 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.2-bin-hadoop3.tgz’

spark-3.5.2-bin-had 100%[===================>] 382.25M  83.9MB/s    in 3.7s    

2024-09-03 02:13:00 (104 MB/s) - ‘spark-3.5.2-bin-hadoop3.tgz’ saved [400822099/400822099]



In [8]:
#Deszipar o spark
!tar xf /content/spark-3.5.2-bin-hadoop3.tgz

In [9]:
#Criar as varia´veis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.2-bin-hadoop3"

In [10]:
#Instalar a lib findspark que ajuda a localizar o Spark no sistema e importá-lo como uma biblioteca regular.
!pip install -q findspark

In [11]:
#Importar a lib findspark
import findspark
findspark.init()

In [12]:
#Criar spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('sparkcolab')\
        .getOrCreate()

In [13]:
#Importar o dataset
from google.colab import files
arquivo = files.upload()

In [14]:
#Criar dataframe
df = spark.read.csv('movies.csv', header=True, inferSchema=True)

In [15]:
df.printSchema()

root
 |-- imdb_title_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- date_published: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- language: string (nullable = true)
 |-- director: string (nullable = true)
 |-- writer: string (nullable = true)
 |-- production_company: string (nullable = true)
 |-- actors: string (nullable = true)
 |-- description: string (nullable = true)
 |-- avg_vote: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- usa_gross_income: string (nullable = true)
 |-- worlwide_gross_income: string (nullable = true)
 |-- metascore: string (nullable = true)
 |-- reviews_from_users: string (nullable = true)
 |-- reviews_from_critics: string (nullable = true)



In [16]:
df.show()

+-------------+--------------------+--------------------+----+--------------+--------------------+--------+----------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-----+----------+----------------+---------------------+---------+------------------+--------------------+
|imdb_title_id|               title|      original_title|year|date_published|               genre|duration|         country|       language|            director|              writer|  production_company|              actors|         description|avg_vote|votes|    budget|usa_gross_income|worlwide_gross_income|metascore|reviews_from_users|reviews_from_critics|
+-------------+--------------------+--------------------+----+--------------+--------------------+--------+----------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-----+----------+-------

In [19]:
#Selecionar colunas
df_movie = df.select('title', 'year', 'country', 'director', 'votes')

In [22]:
df_movie.show()

+--------------------+----+----------------+--------------------+-----+
|               title|year|         country|            director|votes|
+--------------------+----+----------------+--------------------+-----+
|          Miss Jerry|1894|             USA|     Alexander Black|  154|
|The Story of the ...|1906|       Australia|        Charles Tait|  589|
|      Den sorte drøm|1911|Germany, Denmark|           Urban Gad|  188|
|           Cleopatra|1912|             USA|  Charles L. Gaskill|  446|
|           L'Inferno|1911|           Italy|Francesco Bertoli...| 2237|
|From the Manger t...|1912|             USA|       Sidney Olcott|  484|
|      Madame DuBarry|1919|         Germany|      Ernst Lubitsch|  753|
|          Quo Vadis?|1913|           Italy|     Enrico Guazzoni|  273|
|Independenta Roma...|1912|         Romania|Aristide Demetria...|  198|
|         Richard III|1912|     France, USA|André Calmettes, ...|  225|
|            Atlantis|1913|         Denmark|         August Blom

In [23]:
#Mudar tipo das colunas
df_vote = df_movie.withColumn('votos', df_movie['votes'].cast('int')).drop('votes')
df_vote.show()

+--------------------+----+----------------+--------------------+-----+
|               title|year|         country|            director|votos|
+--------------------+----+----------------+--------------------+-----+
|          Miss Jerry|1894|             USA|     Alexander Black|  154|
|The Story of the ...|1906|       Australia|        Charles Tait|  589|
|      Den sorte drøm|1911|Germany, Denmark|           Urban Gad|  188|
|           Cleopatra|1912|             USA|  Charles L. Gaskill|  446|
|           L'Inferno|1911|           Italy|Francesco Bertoli...| 2237|
|From the Manger t...|1912|             USA|       Sidney Olcott|  484|
|      Madame DuBarry|1919|         Germany|      Ernst Lubitsch|  753|
|          Quo Vadis?|1913|           Italy|     Enrico Guazzoni|  273|
|Independenta Roma...|1912|         Romania|Aristide Demetria...|  198|
|         Richard III|1912|     France, USA|André Calmettes, ...|  225|
|            Atlantis|1913|         Denmark|         August Blom

In [25]:
#Filtro
df_vote.filter(df_vote.votos > 500).show()

+--------------------+----+---------+--------------------+-----+
|               title|year|  country|            director|votos|
+--------------------+----+---------+--------------------+-----+
|The Story of the ...|1906|Australia|        Charles Tait|  589|
|           L'Inferno|1911|    Italy|Francesco Bertoli...| 2237|
|      Madame DuBarry|1919|  Germany|      Ernst Lubitsch|  753|
|Fantômas - À l'om...|1913|   France|     Louis Feuillade| 1944|
|Il calvario di un...|1913|   Sweden|     Victor Sjöström|  948|
|Juve contre Fantômas|1913|   France|     Louis Feuillade| 1349|
|     Le mort qui tue|1913|   France|     Louis Feuillade| 1050|
|Lo studente di Praga|1913|  Germany|Paul Wegener, Ste...| 1768|
|    Traffic in Souls|1913|      USA| George Loane Tucker|  552|
|The Avenging Cons...|1914|      USA|       D.W. Griffith| 1105|
|             Cabiria|1914|    Italy|   Giovanni Pastrone| 3073|
|          Cinderella|1914|      USA|      James Kirkwood|  929|
|Fantômas contre F...|191

In [26]:
#Pegar o maximo de votos
from pyspark.sql.functions import max
df_max = df_vote.agg(max('votos').alias('max_votos')).show()

+---------+
|max_votos|
+---------+
|  2278845|
+---------+



In [34]:
#Filtrar o filme com maiores votos
df_max_votos = df_vote.filter(df_vote.votos == 2278845)
df_max_votos.show()

+--------------------+----+-------+--------------+-------+
|               title|year|country|      director|  votos|
+--------------------+----+-------+--------------+-------+
|Le ali della libertà|1994|    USA|Frank Darabont|2278845|
+--------------------+----+-------+--------------+-------+



In [35]:
#Contar paises
from pyspark.sql.functions import count, col, asc, desc
df_sum = df_vote.groupBy('country').count()
df_sum.show()

+--------------------+-----+
|             country|count|
+--------------------+-----+
|           UK, Spain|   27|
|Italy, West Germa...|    1|
|France, Italy, We...|    1|
|Switzerland, Spai...|    1|
|     France, Belarus|    1|
|       France, India|    3|
|    France, Colombia|    2|
|France, UK, Germa...|    1|
|Serbia, Germany, ...|    1|
|Germany, Italy, S...|    1|
|Germany, France, ...|    2|
|   Argentina, Mexico|   10|
|Israel, France, E...|    1|
|   USA, UK, Bulgaria|    2|
|Germany, France, ...|    1|
|France, Belgium, ...|    1|
|France, Israel, G...|    1|
|      India, Germany|    3|
|     USA, France, UK|    6|
|West Germany, Ita...|    1|
+--------------------+-----+
only showing top 20 rows



In [36]:
df_sum.orderBy(col('count').desc()).show()

+------------+-----+
|     country|count|
+------------+-----+
|         USA|28511|
|       India| 6065|
|          UK| 4111|
|       Japan| 3077|
|      France| 3055|
|       Italy| 2444|
|      Canada| 1802|
|     Germany| 1396|
|      Turkey| 1351|
|   Hong Kong| 1239|
|       Spain| 1225|
| South Korea| 1103|
|      Russia|  802|
|   Australia|  785|
|Soviet Union|  766|
|      Brazil|  736|
|     UK, USA|  683|
|      Mexico|  655|
|        Iran|  642|
|      Sweden|  622|
+------------+-----+
only showing top 20 rows



In [37]:
#In
df_vote.select('country', 'title', 'votos').filter(df_vote.country.isin('USA', 'India', 'France')).show()

+-------+--------------------+-----+
|country|               title|votos|
+-------+--------------------+-----+
|    USA|          Miss Jerry|  154|
|    USA|           Cleopatra|  446|
|    USA|From the Manger t...|  484|
| France|Fantômas - À l'om...| 1944|
| France|Juve contre Fantômas| 1349|
| France|     Le mort qui tue| 1050|
|    USA|      Amore di madre|  187|
|    USA|    Traffic in Souls|  552|
|    USA|The Avenging Cons...| 1105|
|    USA|         The Bargain|  144|
|    USA|          Cinderella|  929|
| France|   L'enfant de Paris|  303|
| France|Fantômas contre F...| 1161|
|    USA|A Florida Enchant...|  202|
|    USA|His Majesty, the ...|  415|
|    USA|          Hypocrites|  456|
|    USA|  Judith of Bethulia| 1239|
|    USA|The Patchwork Gir...|  382|
|    USA|The Perils of Pau...|  939|
|    USA|       The Squaw Man|  879|
+-------+--------------------+-----+
only showing top 20 rows



In [43]:
#Not In
df_vote.select('country', 'title', 'votos').filter(~df_vote.country.isin('USA', 'India', 'France')).show()

+----------------+--------------------+-----+
|         country|               title|votos|
+----------------+--------------------+-----+
|       Australia|The Story of the ...|  589|
|Germany, Denmark|      Den sorte drøm|  188|
|           Italy|           L'Inferno| 2237|
|         Germany|      Madame DuBarry|  753|
|           Italy|          Quo Vadis?|  273|
|         Romania|Independenta Roma...|  198|
|     France, USA|         Richard III|  225|
|         Denmark|            Atlantis|  331|
|          Sweden|Il calvario di un...|  948|
|           Italy|Ma l'amor mio non...|  100|
|         Belgium|Maudite soit la g...|  124|
|         Germany|Lo studente di Praga| 1768|
|           Italy|Gli ultimi giorni...|  474|
|           Italy|       Assunta Spina|  378|
|           Italy|             Cabiria| 3073|
|         Germany|           Der Golem| 1040|
|         Denmark|Det hemmelighedsf...|  503|
|     USA, Canada|In the Land of th...|  402|
|         Hungary|            A to

In [46]:
#Utilizando SQL no Pyspark
df_vote.createOrReplaceGlobalTempView('movies')
spark.sql('select country, count(*) as qtd from global_temp.movies GROUP BY country ORDER BY qtd desc').show(truncate=False)

+------------+-----+
|country     |qtd  |
+------------+-----+
|USA         |28511|
|India       |6065 |
|UK          |4111 |
|Japan       |3077 |
|France      |3055 |
|Italy       |2444 |
|Canada      |1802 |
|Germany     |1396 |
|Turkey      |1351 |
|Hong Kong   |1239 |
|Spain       |1225 |
|South Korea |1103 |
|Russia      |802  |
|Australia   |785  |
|Soviet Union|766  |
|Brazil      |736  |
|UK, USA     |683  |
|Mexico      |655  |
|Iran        |642  |
|Sweden      |622  |
+------------+-----+
only showing top 20 rows



In [52]:
#Criando df ratings
df_ratings = spark.read.csv('ratings.csv', header=True, inferSchema=True, sep=',')
df_ratings.show()

+-------------+---------------------+-----------+---------+-----------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------------------------+---------------------+-------------------------+----------------------+-------------------------+----------------------+-------------------------+----------------------+----------------------+-------------------+-------------------+----------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+------------------------+---------------------+---------------------+------------------+----------------------+-------------------+----------------------+-------------------+----------------------+-------------------+---------------------+--------------------+----------------+---------------+--------------------+-------------------+
|imdb_title_id|weighted_average_vote|total_votes|mean_vote|median_vote|votes_10|votes_9|votes_8|votes_7|votes_6|votes_5|vo

In [57]:
#Join entre os DF
df.join(df_ratings, df.imdb_title_id == df_ratings.imdb_title_id, 'inner')\
        .select(df.country, df.title, df.director, df.year, df_ratings.weighted_average_vote, df_ratings.weighted_average_vote)\
        .show(truncate=False)

+----------------+---------------------------------------------------+-------------------------------------+----+---------------------+---------------------+
|country         |title                                              |director                             |year|weighted_average_vote|weighted_average_vote|
+----------------+---------------------------------------------------+-------------------------------------+----+---------------------+---------------------+
|USA             |Miss Jerry                                         |Alexander Black                      |1894|5.9                  |5.9                  |
|Australia       |The Story of the Kelly Gang                        |Charles Tait                         |1906|6.1                  |6.1                  |
|Germany, Denmark|Den sorte drøm                                     |Urban Gad                            |1911|5.8                  |5.8                  |
|USA             |Cleopatra                         